# TensorFlow

In [1]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense

In [2]:
RANDOM_SEED = 32
BOSTON_URL = "http://lib.stat.cmu.edu/datasets/boston"

tf.random.set_seed(RANDOM_SEED)

## Загрузка и обработка данных

In [3]:
df = pd.read_csv(BOSTON_URL, sep="\s+", skiprows=22, header=None)
X = pd.DataFrame(np.hstack([df.values[::2, :], df.values[1::2, :2]]),
                 columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM',
                          'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])
y = df.values[1::2, 2]

X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
dtypes: float64(13)
memory usage: 51.5 KB


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Обучим несколько моделей с разными параметрами

In [6]:
model_params_dict = {
    'epochs': [10, 15],
    'layer_count': [3, 5],
    'layer_shape': [64, 128],
    'activation': ['relu', 'softmax'],
    'optimizer': ['adam', 'ftrl'],
    'loss': ['mse', 'mape'],
    'batch_size': [16, 32]
}

model_params = None
for key, value in model_params_dict.items():
    if model_params is None:
        model_params = pd.DataFrame({key: value})
    else:
        model_params = model_params.merge(pd.DataFrame({key: value}), how='cross')

model_params

,epochs,layer_count,layer_shape,activation,optimizer,loss,batch_size
0,10,3,64,relu,adam,mse,16
1,10,3,64,relu,adam,mse,32
2,10,3,64,relu,adam,mape,16
3,10,3,64,relu,adam,mape,32
4,10,3,64,relu,ftrl,mse,16
...,...,...,...,...,...,...,...
123,15,5,128,softmax,adam,mape,32
124,15,5,128,softmax,ftrl,mse,16
125,15,5,128,softmax,ftrl,mse,32
126,15,5,128,softmax,ftrl,mape,16


In [7]:
%%time
results = {
    'epochs': [],
    'layer_count': [],
    'layer_shape': [],
    'activation': [],
    'optimizer': [],
    'loss': [],
    'batch_size': [],
    'train_loss': [],
    'val_loss': [],
    'mse': [],
    'val_mse': [],
}

for _, params in tqdm(model_params.iterrows(), total=model_params.shape[0]):
    model = Sequential()
    model.add(Input(shape=(13,)))

    for _ in range(params['layer_count']):
        model.add(Dense(params['layer_shape'],
                        activation=params['activation']))

    model.add(Dense(1))
    model.compile(optimizer=params['optimizer'],
                  loss=params['loss'],
                  metrics=['mse'])

    history = model.fit(X_train, y_train,
                        epochs=params['epochs'],
                        batch_size=params['batch_size'],
                        validation_data=(X_test, y_test), 
                        verbose=0)

    for key in model_params_dict.keys():
        results[key].append(params[key])
    
    results['train_loss'].append(history.history['loss'][-1])
    results['val_loss'].append(history.history['val_loss'][-1])
    results['mse'].append(history.history['mse'][-1])
    results['val_mse'].append(history.history['val_mse'][-1])

  0%|          | 0/128 [00:00<?, ?it/s]

CPU times: total: 3min 57s
Wall time: 3min 17s


In [8]:
results = pd.DataFrame(results)
results.sort_values('val_mse', ascending=True).head(15)

,epochs,layer_count,layer_shape,activation,optimizer,loss,batch_size,train_loss,val_loss,mse,val_mse
96,15,5,64,relu,adam,mse,16,11.063757,14.286579,11.063757,14.286579
113,15,5,128,relu,adam,mse,32,10.132361,14.409283,10.132361,14.409283
16,10,3,128,relu,adam,mse,16,13.614008,15.367763,13.614008,15.367763
64,15,3,64,relu,adam,mse,16,12.589449,15.954492,12.589449,15.954492
112,15,5,128,relu,adam,mse,16,8.143099,16.017178,8.143099,16.017178
80,15,3,128,relu,adam,mse,16,10.247164,16.122231,10.247164,16.122231
97,15,5,64,relu,adam,mse,32,14.804042,16.673729,14.804042,16.673729
82,15,3,128,relu,adam,mape,16,10.148890,13.021497,12.625764,16.960331
32,10,5,64,relu,adam,mse,16,14.291532,17.134394,14.291532,17.134394
0,10,3,64,relu,adam,mse,16,15.861097,17.272928,15.861097,17.272928


## Обучим лучшую модель

In [9]:
epochs = 15
layer_count = 5
layer_shape = 64
activation = 'relu'
optimizer = 'adam'
loss = 'mse'
batch_size = 16

model = Sequential([
    Input(shape=(13,)),
    *[Dense(layer_shape, activation=activation) for _ in range(layer_count)],
    Dense(1),
])

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['mse']
)

model.fit(X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
)

y_pred = model.predict(X_test)
print('r2_mape =', round(r2_score(y_test, y_pred), 2))

Epoch 1/15
23/23 [==============================] - 0s 9ms/step - loss: 511.6479 - mse: 511.6479 - val_loss: 249.4464 - val_mse: 249.4464
Epoch 2/15
23/23 [==============================] - 0s 6ms/step - loss: 124.2650 - mse: 124.2650 - val_loss: 50.9579 - val_mse: 50.9579
Epoch 3/15
23/23 [==============================] - 0s 6ms/step - loss: 45.5795 - mse: 45.5795 - val_loss: 31.2854 - val_mse: 31.2854
Epoch 4/15
23/23 [==============================] - 0s 6ms/step - loss: 24.6660 - mse: 24.6660 - val_loss: 22.6164 - val_mse: 22.6164
Epoch 5/15
23/23 [==============================] - 0s 6ms/step - loss: 19.4683 - mse: 19.4683 - val_loss: 19.7835 - val_mse: 19.7835
Epoch 6/15
23/23 [==============================] - 0s 5ms/step - loss: 16.2310 - mse: 16.2310 - val_loss: 17.8334 - val_mse: 17.8334
Epoch 7/15
23/23 [==============================] - 0s 5ms/step - loss: 14.0818 - mse: 14.0818 - val_loss: 16.9594 - val_mse: 16.9594
Epoch 8/15
23/23 [==============================] - 0s 5